#**Introduction to Financial Python**


##Pandas-Resampling and DataFrame

**Introduction**

In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.


**Fetching Data**

Here we use the Quandl API to retrieve data. 

In [8]:
pip install quandl

To use quandl appropiately you may need your own API key, for that kind of resource, you have to get Quandl account.

In [12]:
quandl.ApiConfig.api_key = 'ivsLh16Y4q-1WPFL8EM3'

We will create a Series named "aapl" whose values are Apple's daily closing prices, which are of course indexed by dates: 

In [14]:
aapl_table = quandl.get('WIKI/AAPL')
aapl = aapl_table['Adj. Close']['2017']
print(aapl) 

Date
2017-01-03    114.715378
2017-01-04    114.586983
2017-01-05    115.169696
2017-01-06    116.453639
2017-01-09    117.520300
                 ...    
2017-12-22    175.010000
2017-12-26    170.570000
2017-12-27    170.600000
2017-12-28    171.080000
2017-12-29    169.230000
Name: Adj. Close, Length: 249, dtype: float64


Recall that we can fetch a specific data point using `series['yyyy-mm-dd']`. We can also fetch the data in a specific month using `series['yyyy-mm']`. 

In [15]:
print(aapl['2017-3'])

Date
2017-03-01    138.657681
2017-03-02    137.834404
2017-03-03    138.647762
2017-03-06    138.211326
2017-03-07    138.389868
2017-03-08    137.874080
2017-03-09    137.556672
2017-03-10    138.012946
2017-03-13    138.072460
2017-03-14    137.864161
2017-03-15    139.322254
2017-03-16    139.550391
2017-03-17    138.856061
2017-03-20    140.314154
2017-03-21    138.707276
2017-03-22    140.274478
2017-03-23    139.778528
2017-03-24    139.500796
2017-03-27    139.738852
2017-03-28    142.635200
2017-03-29    142.952608
2017-03-30    142.764147
2017-03-31    142.496334
Name: Adj. Close, dtype: float64


Or in several consecutive months: 

In [16]:
aapl['2017-2':'2017-4']

Date
2017-02-01    127.159749
2017-02-02    126.942467
2017-02-03    127.485673
2017-02-06    128.680728
2017-02-07    129.905412
                 ...    
2017-04-24    142.476496
2017-04-25    143.369205
2017-04-26    142.487208
2017-04-27    142.625281
2017-04-28    142.486415
Name: Adj. Close, Length: 61, dtype: float64

`.head(N)` and `.tail(N)` are methods for quickly accessing the first or last N elements. 

In [17]:
print(aapl.head()) 
print(aapl.tail(10)) 

Date
2017-01-03    114.715378
2017-01-04    114.586983
2017-01-05    115.169696
2017-01-06    116.453639
2017-01-09    117.520300
Name: Adj. Close, dtype: float64
Date
2017-12-15    173.87
2017-12-18    176.42
2017-12-19    174.54
2017-12-20    174.35
2017-12-21    175.01
2017-12-22    175.01
2017-12-26    170.57
2017-12-27    170.60
2017-12-28    171.08
2017-12-29    169.23
Name: Adj. Close, dtype: float64


**Resampling**

`series.resample(freq)` is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

`series.resample.mean()` is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean: 

In [18]:
by_month = aapl.resample('M').mean()
print(by_month) 

Date
2017-01-31    118.093136
2017-02-28    132.456268
2017-03-31    139.478802
2017-04-30    141.728436
2017-05-31    151.386305
2017-06-30    147.233064
2017-07-31    147.706190
2017-08-31    158.856375
2017-09-30    157.606500
2017-10-31    157.811627
2017-11-30    172.214500
2017-12-31    171.893100
Freq: M, Name: Adj. Close, dtype: float64


We can also aggregate the data by week: 

In [19]:
by_week = aapl.resample('W').mean()
print(by_week.head()) 

Date
2017-01-08    115.231424
2017-01-15    117.755360
2017-01-22    118.461035
2017-01-29    119.667448
2017-02-05    124.313346
Freq: W-SUN, Name: Adj. Close, dtype: float64


We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day. 

In [20]:
three_day = aapl.resample('3D').mean()
two_week  = aapl.resample('2W').mean()
two_month = aapl.resample('2M').mean()

Besides the `mean()` method, other methods can also be used with the resampler: 

In [21]:
std = aapl.resample('W').std()    # standard deviation
max = aapl.resample('W').max()    # maximum value
min = aapl.resample('W').min()    # minimum value

Often we want to calculate monthly returns of a stock, based on prices on the last day of each month. To fetch those prices, we use the `series.resample.agg()` method: 

In [22]:
last_day = aapl.resample('M').agg(lambda x: x[-1])
print(last_day) 

Date
2017-01-31    119.851150
2017-02-28    135.880362
2017-03-31    142.496334
2017-04-30    142.486415
2017-05-31    152.142689
2017-06-30    143.438008
2017-07-31    148.248489
2017-08-31    164.000000
2017-09-30    154.120000
2017-10-31    169.040000
2017-11-30    171.850000
2017-12-31    169.230000
Freq: M, Name: Adj. Close, dtype: float64


Or directly calculate the monthly rates of return using the data for the first day and the last day: 

In [23]:
monthly_return = aapl.resample('M').agg(lambda x: x[-1]/x[1] - 1)
print(monthly_return) 

Date
2017-01-31    0.045940
2017-02-28    0.070409
2017-03-31    0.033823
2017-04-30   -0.007736
2017-05-31    0.039829
2017-06-30   -0.073528
2017-07-31    0.033035
2017-08-31    0.047890
2017-09-30   -0.049112
2017-10-31    0.094252
2017-11-30    0.022247
2017-12-31   -0.003357
Freq: M, Name: Adj. Close, dtype: float64


Series object also provides us some convenient methods to do some quick calculation. 

In [24]:
print(monthly_return.mean()) 
print(monthly_return.std()) 
print(monthly_return.max()) 

0.02114094011940022
0.04775652864223314
0.09425168306576914


Another two methods frequently used on Series are .diff() and .pct_change(). The former calculates the difference between consecutive elements, and the latter calculates the percentage change. 

In [25]:
print(last_day.diff()) 
print(last_day.pct_change()) 

Date
2017-01-31          NaN
2017-02-28    16.029211
2017-03-31     6.615972
2017-04-30    -0.009919
2017-05-31     9.656274
2017-06-30    -8.704681
2017-07-31     4.810482
2017-08-31    15.751511
2017-09-30    -9.880000
2017-10-31    14.920000
2017-11-30     2.810000
2017-12-31    -2.620000
Freq: M, Name: Adj. Close, dtype: float64
Date
2017-01-31         NaN
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0: 

In [26]:
daily_return = last_day.pct_change()
print(daily_return.fillna(0)) 

Date
2017-01-31    0.000000
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


Alternatively, we can fill a NaN with the next fitted value. This is called 'backward fill', or 'bfill' in short: 

In [27]:
daily_return = last_day.pct_change()
print(daily_return.fillna(method = 'bfill')) 

Date
2017-01-31    0.133743
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


 As expected, since there is a 'backward fill' method, there must be a 'forward fill' method, or 'ffill' in short. However we can't use it here because the NaN is the first value.

We can also simply remove NaN values by `.dropna() `

In [28]:
daily_return = last_day.pct_change().dropna()
print(daily_return) 

Date
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


##DataFrame

The DataFrame is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.

**Create DataFrames**

The most common method of creating a DataFrame is passing a dictionary: 

In [30]:
import pandas as pd

dict = {'AAPL': [143.5,  144.09, 142.73, 144.18, 143.77],
        'GOOG': [898.7,  911.71, 906.69, 918.59, 926.99],
        'IBM':  [155.58, 153.67, 152.36, 152.94, 153.49]}
dates = pd.date_range('2017-07-03', periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = dates)
print(df) 

              AAPL    GOOG     IBM
2017-07-03  143.50  898.70  155.58
2017-07-04  144.09  911.71  153.67
2017-07-05  142.73  906.69  152.36
2017-07-06  144.18  918.59  152.94
2017-07-07  143.77  926.99  153.49


**Manipulating DataFrames**

We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: df['column_name']

If a column name contains no spaces, then we can also use df.column_name to fetch a column: 

In [31]:
df = aapl_table
print(df.Close.tail(5)) 
print(df['Adj. Volume'].tail(5)) 

Date
2018-03-21    171.270
2018-03-22    168.845
2018-03-23    164.940
2018-03-26    172.770
2018-03-27    168.340
Name: Close, dtype: float64
Date
2018-03-21    35247358.0
2018-03-22    41051076.0
2018-03-23    40248954.0
2018-03-26    36272617.0
2018-03-27    38962839.0
Name: Adj. Volume, dtype: float64


All the methods we applied to a Series index such as `iloc[]`, `loc[]` and resampling methods, can also be applied to a DataFrame: 

In [32]:
aapl_2016 = df['2016']
aapl_month = aapl_2016.resample('M').agg(lambda x: x[-1])
print(aapl_month) 

              Open      High     Low  ...    Adj. Low  Adj. Close  Adj. Volume
Date                                  ...                                     
2016-01-31   94.79   97.3400   94.35  ...   91.156128   94.044912   64416504.0
2016-02-29   96.86   98.2300   96.65  ...   93.880927   93.919781   35216277.0
2016-03-31  109.72  109.9000  108.88  ...  105.760531  105.867380   25888449.0
2016-04-30   93.99   94.7200   92.51  ...   89.859540   91.054300   68531478.0
2016-05-31   99.60  100.4000   98.82  ...   96.575559   97.591939   42307212.0
2016-06-30   94.44   95.7700   94.30  ...   92.158220   93.428693   35836356.0
2016-07-31  104.19  104.5500  103.68  ...  101.325177  101.843140   27733688.0
2016-08-31  105.66  106.5699  105.64  ...  103.796505  104.248477   29662406.0
2016-09-30  112.46  113.3700  111.80  ...  109.849008  111.077195   36379106.0
2016-10-31  113.65  114.2300  113.20  ...  111.224577  111.558644   26419398.0
2016-11-30  111.56  112.2000  110.27  ...  108.90800

We may select certain columns of a DataFrame using their names: 

In [34]:
aapl_bar = aapl_month[['Open', 'High', 'Low', 'Close']]
print(aapl_bar) 

              Open      High     Low   Close
Date                                        
2016-01-31   94.79   97.3400   94.35   97.34
2016-02-29   96.86   98.2300   96.65   96.69
2016-03-31  109.72  109.9000  108.88  108.99
2016-04-30   93.99   94.7200   92.51   93.74
2016-05-31   99.60  100.4000   98.82   99.86
2016-06-30   94.44   95.7700   94.30   95.60
2016-07-31  104.19  104.5500  103.68  104.21
2016-08-31  105.66  106.5699  105.64  106.10
2016-09-30  112.46  113.3700  111.80  113.05
2016-10-31  113.65  114.2300  113.20  113.54
2016-11-30  111.56  112.2000  110.27  110.52
2016-12-31  116.65  117.2000  115.43  115.82


We can even specify both rows and columns using loc[]. The row indices and column names are separated by a comma:

In [35]:
print(aapl_month.loc['2016-03':'2016-06', ['Open', 'High', 'Low', 'Close']]) 

              Open    High     Low   Close
Date                                      
2016-03-31  109.72  109.90  108.88  108.99
2016-04-30   93.99   94.72   92.51   93.74
2016-05-31   99.60  100.40   98.82   99.86
2016-06-30   94.44   95.77   94.30   95.60


The subset methods in DataFrame is quite useful. By writing logical statements in square brackets, we can make customized subsets: 

In [37]:
import numpy as np

above = aapl_bar[aapl_bar.Close > np.mean(aapl_bar.Close)]
print(above) 

              Open      High     Low   Close
Date                                        
2016-03-31  109.72  109.9000  108.88  108.99
2016-08-31  105.66  106.5699  105.64  106.10
2016-09-30  112.46  113.3700  111.80  113.05
2016-10-31  113.65  114.2300  113.20  113.54
2016-11-30  111.56  112.2000  110.27  110.52
2016-12-31  116.65  117.2000  115.43  115.82


**Data Validation**

As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame: 

In [38]:
aapl_bar['rate_return'] = aapl_bar.Close.pct_change()
print(aapl_bar) 

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34          NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. isnull() provides a convenient way to check abnormal values. 

In [40]:
missing = aapl_bar.isnull()
print(missing) 
print('---------------------------------------------') 
print(missing.describe()) 

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True
2016-02-29  False  False  False  False        False
2016-03-31  False  False  False  False        False
2016-04-30  False  False  False  False        False
2016-05-31  False  False  False  False        False
2016-06-30  False  False  False  False        False
2016-07-31  False  False  False  False        False
2016-08-31  False  False  False  False        False
2016-09-30  False  False  False  False        False
2016-10-31  False  False  False  False        False
2016-11-30  False  False  False  False        False
2016-12-31  False  False  False  False        False
---------------------------------------------
         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12    

The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily: 

In [41]:
print(missing[missing.rate_return == True]) 

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method `dropna()` and `fillna()` can be applied to a DataFrame. 

In [42]:
drop = aapl_bar.dropna()
print(drop) 
print('\n--------------------------------------------------\n') 
fill = aapl_bar.fillna(0)
print(fill) 

              Open      High     Low   Close  rate_return
Date                                                     
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955

--------------------------------------------------

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34     0.000000
2016-02-29   96.86 

**DataFrame Concat**

We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function `concat()` allows us to merge multiple Series into a DataFrame: 

In [43]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1, s2], axis = 1)
print(data_frame) 

     AAPL    GOOG
0  143.50  898.70
1  144.09  911.71
2  142.73  906.69
3  144.18  918.59
4  143.77  926.99


The "axis = 1" parameter will join two DataFrames by columns:

In [44]:
log_price = np.log(aapl_bar.Close)
log_price.name = 'log_price'
print(log_price) 
print('\n--------------------------------------------\n') 
concat = pd.concat([aapl_bar, log_price], axis = 1)
print(concat) 

Date
2016-01-31    4.578210
2016-02-29    4.571510
2016-03-31    4.691256
2016-04-30    4.540525
2016-05-31    4.603769
2016-06-30    4.560173
2016-07-31    4.646408
2016-08-31    4.664382
2016-09-30    4.727830
2016-10-31    4.732155
2016-11-30    4.705197
2016-12-31    4.752037
Freq: M, Name: log_price, dtype: float64

--------------------------------------------

              Open      High     Low   Close  rate_return  log_price
Date                                                                
2016-01-31   94.79   97.3400   94.35   97.34          NaN   4.578210
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678   4.571510
2016-03-31  109.72  109.9000  108.88  108.99     0.127211   4.691256
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921   4.540525
2016-05-31   99.60  100.4000   98.82   99.86     0.065287   4.603769
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660   4.560173
2016-07-31  104.19  104.5500  103.68  104.21     0.090063   4.646408
2016-08-31

We can also join two DataFrames by rows. Consider these two DataFrames: 

In [45]:
df_volume = aapl_table.loc['2016-10':'2017-04', ['Volume', 'Split Ratio']].resample('M').agg(lambda x: x[-1])
print(df_volume) 
print('\n-------------------------------------------\n') 
df_2017 = aapl_table.loc['2016-10':'2017-04', ['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print(df_2017) 

                Volume  Split Ratio
Date                               
2016-10-31  26419398.0          1.0
2016-11-30  36162258.0          1.0
2016-12-31  30586265.0          1.0
2017-01-31  49200993.0          1.0
2017-02-28  23482860.0          1.0
2017-03-31  19661651.0          1.0
2017-04-30  20247187.0          1.0

-------------------------------------------

              Open     High     Low   Close
Date                                       
2016-10-31  113.65  114.230  113.20  113.54
2016-11-30  111.56  112.200  110.27  110.52
2016-12-31  116.65  117.200  115.43  115.82
2017-01-31  121.15  121.390  120.62  121.35
2017-02-28  137.08  137.435  136.70  136.99
2017-03-31  143.72  144.270  143.01  143.66
2017-04-30  144.09  144.300  143.27  143.65


Now we merge the DataFrames with our DataFrame 'aapl_bar' 

In [46]:
concat = pd.concat([aapl_bar, df_volume], axis = 1)
print(concat) 

              Open      High     Low  ...  rate_return      Volume  Split Ratio
Date                                  ...                                      
2016-01-31   94.79   97.3400   94.35  ...          NaN         NaN          NaN
2016-02-29   96.86   98.2300   96.65  ...    -0.006678         NaN          NaN
2016-03-31  109.72  109.9000  108.88  ...     0.127211         NaN          NaN
2016-04-30   93.99   94.7200   92.51  ...    -0.139921         NaN          NaN
2016-05-31   99.60  100.4000   98.82  ...     0.065287         NaN          NaN
2016-06-30   94.44   95.7700   94.30  ...    -0.042660         NaN          NaN
2016-07-31  104.19  104.5500  103.68  ...     0.090063         NaN          NaN
2016-08-31  105.66  106.5699  105.64  ...     0.018136         NaN          NaN
2016-09-30  112.46  113.3700  111.80  ...     0.065504         NaN          NaN
2016-10-31  113.65  114.2300  113.20  ...     0.004334  26419398.0          1.0
2016-11-30  111.56  112.2000  110.27  ..

By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join': 

In [47]:
concat = pd.concat([aapl_bar, df_volume], axis = 1, join = 'inner')
print(concat) 

              Open    High     Low  ...  rate_return      Volume  Split Ratio
Date                                ...                                      
2016-10-31  113.65  114.23  113.20  ...     0.004334  26419398.0          1.0
2016-11-30  111.56  112.20  110.27  ...    -0.026599  36162258.0          1.0
2016-12-31  116.65  117.20  115.43  ...     0.047955  30586265.0          1.0

[3 rows x 7 columns]


Only the intersection part was left if use 'inner join' method. Now let's try to append a DataFrame to another one: 

In [48]:
append = aapl_bar.append(df_2017)
print(append) 

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34          NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955
2016-10-31  113.65  114.2300  113.20  113.54          NaN
2016-11-30  111.56  112.2000  110.27  110.52          NaN
2016-12-31  116.65  117.2000  115.43  115.82          NaN
2017-01-31  12

'Append' is essentially to concat two DataFrames by axis = 0, thus here is an alternative way to append: 

In [49]:
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat) 

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34          NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955
2016-10-31  113.65  114.2300  113.20  113.54          NaN
2016-11-30  111.56  112.2000  110.27  110.52          NaN
2016-12-31  116.65  117.2000  115.43  115.82          NaN
2017-01-31  12

Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here: 

In [50]:
df_2017.columns = ['Change', 'High', 'Low', 'Close']
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat) 

              Open      High     Low   Close  rate_return  Change
Date                                                             
2016-01-31   94.79   97.3400   94.35   97.34          NaN     NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678     NaN
2016-03-31  109.72  109.9000  108.88  108.99     0.127211     NaN
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921     NaN
2016-05-31   99.60  100.4000   98.82   99.86     0.065287     NaN
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660     NaN
2016-07-31  104.19  104.5500  103.68  104.21     0.090063     NaN
2016-08-31  105.66  106.5699  105.64  106.10     0.018136     NaN
2016-09-30  112.46  113.3700  111.80  113.05     0.065504     NaN
2016-10-31  113.65  114.2300  113.20  113.54     0.004334     NaN
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599     NaN
2016-12-31  116.65  117.2000  115.43  115.82     0.047955     NaN
2016-10-31     NaN  114.2300  113.20  113.54          NaN  113.65
2016-11-30

Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'. 

##Summary

Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the Pandas official documentations for help.
